In [1]:
import re
import time
import random
import pickle
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import namedtuple, deque

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from unityagents import UnityEnvironment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
PATH = "/Volumes/BC_Clutch/Dropbox/DeepRLND/rl_navigation/"
APP_PATH = PATH + "data/Banana.app"
CHART_PATH = PATH + "charts/"
CHECKPOINT_PATH = PATH + "models/"

In [3]:
# env = UnityEnvironment(APP_PATH)

In [4]:
# brain_name = env.brain_names[0]
# brain = env.brains[brain_name]
# env_info = env.reset(True)[brain_name]

In [5]:
# print(f"Number of agents: {len(env_info.agents)}")
# print(f"Number of actions: {brain.vector_action_space_size}")
# print(f"Number of states: {len(env_info.vector_observations[0])}")

In [6]:
LR = 1e-4
GAMMA = 0.97
TAU = 1e-3
BUFFER_SIZE = int(1e5)
BATCH_SIZE = 64
SEED = 0
UPDATE_EVERY = 4

In [7]:
class QNetwork(nn.Module):
    """Actor (Policy) Model."""
    
    def __init__(self, state_size, action_size, seed, hidden=[64,64]):
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, hidden[0])
        self.fc2 = nn.Linear(hidden[0],hidden[1])
        self.fc3 = nn.Linear(hidden[1], action_size)
        
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
class ReplayBuffer:
    """
    Fixed-size buffer to store experience tuples.
    """
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience",field_names=["state","action","reward","next_state","done"])
        self.seed = random.seed(seed)
        
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
        
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)
        
    def __len__(self):
        """Return current size of internal memory."""
        return len(self.memory)
    
class Vanilla:
    """Initialize the base agent class."""
    
    def __init__(self, state_size, action_size, seed,buffer_size=BUFFER_SIZE,
                 batch_size=BATCH_SIZE, gamma=GAMMA,lr=LR,update_every=UPDATE_EVERY):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        self.buffer_size = int(buffer_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.lr = lr
        self.update_every = update_every
        
        # Q Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(),lr=LR)
        
        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        self.t_step = 0
        
    def step(self, state, action, reward, next_state, done):
        # save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # learn every UPDATE EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # if enough samples available in memory, get random subset and learn
            if len(self.memory)>BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)
                
    def act(self, state, eps=0.):
        """Return actions for given state as per current policy."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        
        # sets the pytorch module into evaluation mode
        self.qnetwork_local.eval()
        
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
        
        # epsilon greedy action selection
        if random.random()>eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        
    def learn(self, experiences, gamma):
        """Update value params given batch of experience tuples."""
        states, actions, rewards, next_states, dones = experiences
        
        # get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # compute Q targets for current states
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        # compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # minimize loss
        self.optimizer.zero_grad()
        loss.backwward()
        self.optimizer.step()
        
        # update target network
        self.soft_update(self.qnetwork_local, self.qnetwork_target,TAU)
        
    def soft_update(self, local_model, target_model, tau):
        """
        Soft update model params.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        """
        for target_param, local_param in zip(target_model.parameters(),local_model.parameters()):
            target_param.data.copy_(tau*local_param.data+(1.0-tau)*target_param.data)
            
# class DeepQNetwork:
#     """Deep Q Learning Network."""
#     def __init__(self, PATH, model_name, n_episodes, max_t, eps_start, eps_end, eps_decay, seed):
#         self.env = UnityEnvironment(PATH + "data/Banana.app")
#         self.brain_name = self.env.brain_names[0]
#         self.brain = self.env.brains[self.brain_name]
#         self.env_info = self.env.reset(True)[self.brain_name]
#         self.scores = []
#         self.scores_window = deque(maxlen=100)
#         self.eps = eps_start
#         self.state = self.env_info.vector_observations[0]
#         self.state_size = len(self.state)
#         self.action_size = self.brain.vector_action_space_size
#         self.seed = seed
#         self.agent = Vanilla(self.state_size, self.action_size, self.seed)
#         self.timestamp = re.sub(r"\D","",str(datetime.datetime.now()))[:12]
        
#     def train(self, n_episodes, max_t, state_size, action_size, seed):
#         start = time.time()
#         for i_episode in range(1, n_episodes+1):
#             state = self.env.reset()
#             score = 0
#             for t in range(max_t):
#                 state = self.env_info.vector_observations[0]
#                 action = self.agent.act(self.state,self.eps)
#                 env_info = self.env.step(action)[self.brain_name]
#                 next_state = self.env_info.vector_observations[0]
#                 reward = self.env_info.rewards[0]
#                 done = self.env_info.local_done[0]
#                 self.agent.step(state, action, reward, next_state, done)
#                 score += reward
#                 state = next_state
#                 if done:
#                     break
#             scores_window.append(score)
#             scores.append(score)
#             self.eps = max(eps_end, eps_decay*eps)
#             print(f"Episode {i_episode}\tAverage Score: {np.mean(scores_window):.2f}")
#             if i_episode % 100 == 0:
#                 print(f"Episode {i_episode}\tAverage Score {np.mean(scores_window):.2f}")
#             if np.mean(scores_window)>=score_target:
#                 print(f"Environment solved in {i_episode:d} episodes.\tAverage Score: {np.mean(scores_window):.2f}")
#                 checkpath = PATH + f"models/checkpoint-{model_name}-{timestamp}.pth"
#                 torch.save(agent.qnetwork_local.state_dict(),checkpath)
#                 print(f"Checkpoint saved at {checkpath}")
#                 break
#         end = time.time()
#         result_dict = {
#                         "scores":scores,
#                         "clocktime":round((end-start)/60,2),
#                         "state_size":state_size,
#                         "action_size":action_size,
#                         "seed":seed,
#                         "n_episodes":n_episodes,
#                         "max_t":max_t,
#                         "eps_start":eps_start,
#                         "eps_end":eps_end,
#                         "eps_decay":eps_decay,
#                         "train_mode":train_mode
#                         }
#         pklpath = PATH + f"charts/ResultDict-{timestamp}.pkl"
#         with open(pklpath,'wb') as handle:
#             pickle.dump(result_dict,handle)
#         print(f"Scores pickled at {pklpath}")
#         return result_dict

In [8]:
def dqn(model_name, n_episodes, max_t, eps_start, eps_end, eps_decay,train_mode):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    env_info = env.reset(train_mode)[brain_name]
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start
    print("Loading monkey.")
    print("Monkey training on bananas.")
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            state = env_info.vector_observations[0]            # get the current state
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            score += reward                                # update the score
            state = next_state                             # roll over the state to next time step
            if done:                                       # exit loop if episode finished
                break
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=score_target:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            checkpath = CHECKPOINT_PATH + f'checkpoint-{model_name}-{timestamp}.pth'
            torch.save(agent.qnetwork_local.state_dict(), checkpath)
            print(f"Checkpoint saved at {checkpath}")
            break
    return scores

In [9]:
model_name = "Vanilla"
n_episodes = 1000
max_t = 1000
eps_start = 0.4
eps_end = 0.01
eps_decay = 0.995
state_size = 37
action_size = 4
seed = 0
train_mode = True

score_target = 0.0

timestamp = re.sub(r"\D","",str(datetime.datetime.now()))[:12]

env = UnityEnvironment(file_name=APP_PATH)

brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset()[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [10]:
# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space
state = env_info.vector_observations[0]
print('States look like:\n', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like:
 [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


In [11]:

# agent = Vanilla(state_size, action_size, seed)
# scores = dqn(model_name, n_episodes, max_t, eps_start, eps_end, eps_decay,train_mode=True)

# timestamp = re.sub(r"\D","",str(datetime.datetime.now()))[:12]

# dqn = DeepQNetwork(PATH, model_name, n_episodes, max_t, eps_start, eps_end, eps_decay, seed)

# result_dict = dqn.train(n_episodes, max_t, state_size, action_size, seed)

# result_dict = DeepQNetwork.train(n_episodes, max_t, state_size, action_size, seed, timestamp)

In [12]:
# train agent
result_dict = {}
function_list = [
                 ("Vanilla",Vanilla(state_size, action_size, seed)),
#                  ("Double", Double(state_size, action_size, seed)),
#                  ("PrioritizedReplay",PrioritizedReplay(state_size, action_size, seed))
                ]

for function in function_list:
    start = time.time()
    name = function[0]
    agent = function[1]
    print(f"**{name}**")
    scores = dqn(name, n_episodes, max_t, eps_start, eps_end, eps_decay,train_mode)
    end = time.time()
    result_dict[name] = {
                    "scores":scores,
                    "clocktime":round((end-start)/60,2),
                    "state_size":state_size,
                    "action_size":action_size,
                    "seed":seed,
                    "n_episodes":n_episodes,
                    "max_t":max_t,
                    "eps_start":eps_start,
                    "eps_end":eps_end,
                    "eps_decay":eps_decay,
                    "train_mode":train_mode
                    }

pklpath = CHART_PATH + f"ResultDict-{timestamp}.pkl"
with open(pklpath, 'wb') as handle:
    pickle.dump(result_dict, handle)
print(f"Scores pickled at {pklpath}")

**Vanilla**
Loading monkey.
Monkey training on bananas.


RuntimeError: Expected object of type torch.LongTensor but found type torch.FloatTensor for argument #3 'index'